In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from sklearn import preprocessing
import tensorflow as tf

In [2]:
data_train = pd.read_csv('../train_data.csv').values
data_test = pd.read_csv('../test_data.csv').values

# Build X and y
X_train = data_train[:, :-1]
y_train = data_train[:, -1]
X_test = data_test[:, :-1]
y_test = data_test[:, -1]

# Number of stocks in training data
n_att = X_train.shape[1]

# Neurons
n_neurons_1 = 1024
n_neurons_2 = 512
n_neurons_3 = 256
n_neurons_4 = 128

# Placeholder
X = tf.placeholder(dtype=tf.float32, shape=[None, n_att])
Y = tf.placeholder(dtype=tf.float32, shape=[None])

# Initializers
sigma = 1
weight_initializer = tf.variance_scaling_initializer(mode="fan_avg", distribution="uniform", scale=sigma)
bias_initializer = tf.zeros_initializer()

# Hidden weights
W_hidden_1 = tf.Variable(weight_initializer([n_att, n_neurons_1]))
bias_hidden_1 = tf.Variable(bias_initializer([n_neurons_1]))
W_hidden_2 = tf.Variable(weight_initializer([n_neurons_1, n_neurons_2]))
bias_hidden_2 = tf.Variable(bias_initializer([n_neurons_2]))
W_hidden_3 = tf.Variable(weight_initializer([n_neurons_2, n_neurons_3]))
bias_hidden_3 = tf.Variable(bias_initializer([n_neurons_3]))
W_hidden_4 = tf.Variable(weight_initializer([n_neurons_3, n_neurons_4]))
bias_hidden_4 = tf.Variable(bias_initializer([n_neurons_4]))

# Output weights
W_out = tf.Variable(weight_initializer([n_neurons_4, 1]))
bias_out = tf.Variable(bias_initializer([1]))

# Hidden layer
hidden_1 = tf.nn.relu(tf.add(tf.matmul(X, W_hidden_1), bias_hidden_1))
hidden_2 = tf.nn.relu(tf.add(tf.matmul(hidden_1, W_hidden_2), bias_hidden_2))
hidden_3 = tf.nn.relu(tf.add(tf.matmul(hidden_2, W_hidden_3), bias_hidden_3))
hidden_4 = tf.nn.relu(tf.add(tf.matmul(hidden_3, W_hidden_4), bias_hidden_4))

# Output layer (transpose!)
out = tf.transpose(tf.add(tf.matmul(hidden_4, W_out), bias_out))

# Cost function
mse = tf.reduce_mean(tf.squared_difference(out, Y))

# Optimizer
opt = tf.train.AdamOptimizer().minimize(mse)

# Saver
saver = tf.train.Saver()

In [3]:
with tf.Session() as sess:
    
    # Init
    sess.run(tf.global_variables_initializer())

    # Fit neural net
    batch_size = 256
    mse_train = []
    mse_test = []

    # Run
    epochs = 2
    for e in range(epochs):

        # Shuffle training data
        shuffle_indices = np.random.permutation(np.arange(len(y_train)))
        X_train = X_train[shuffle_indices]
        y_train = y_train[shuffle_indices]

        # Minibatch training
        for i in range(0, len(y_train) // batch_size):
            start = i * batch_size
            batch_x = X_train[start:start + batch_size]
            batch_y = y_train[start:start + batch_size]
            # Run optimizer with batch
            sess.run(opt, feed_dict={X: batch_x, Y: batch_y})

            # Show progress
            if np.mod(i, 50) == 0:
                # MSE train and test
                mse_train.append(sess.run(mse, feed_dict={X: X_train, Y: y_train}))
                mse_test.append(sess.run(mse, feed_dict={X: X_test, Y: y_test}))
                print('MSE Train: ', mse_train[-1])
                print('MSE Test: ', mse_test[-1])
                # Prediction
                pred = sess.run(out, feed_dict={X: X_test})
                print('pred: ', pred)
    save_path = saver.save(sess, "../models/model.ckpt")
    print("Model saved in path: %s" % save_path)

MSE Train:  0.86318576
MSE Test:  0.8708358
pred:  [[-0.06295124 -0.3255412  -0.0158576  ... -0.40194136 -0.08712311
  -0.14461745]]
MSE Train:  0.23197292
MSE Test:  0.23305672
pred:  [[ 1.4949491  -0.4428597   2.1428452  ... -0.3987369  -0.3555596
   0.22368476]]
MSE Train:  0.23459873
MSE Test:  0.23593776
pred:  [[ 1.2818843  -0.522071    1.5890675  ... -0.44863433 -0.42052764
  -0.2940093 ]]
MSE Train:  0.23434186
MSE Test:  0.23590441
pred:  [[ 1.6301413  -0.34406024  1.8532928  ... -0.3182347  -0.415583
   0.42648458]]
MSE Train:  0.21211544
MSE Test:  0.21123993
pred:  [[ 1.2216898  -0.43794635  1.9278084  ... -0.46825606 -0.43444514
  -0.11658355]]
MSE Train:  0.20582218
MSE Test:  0.20778863
pred:  [[ 1.4797482  -0.40469182  2.37842    ... -0.42214894 -0.44224334
   0.01275035]]
MSE Train:  0.20697708
MSE Test:  0.20876266
pred:  [[ 1.3511792  -0.46143353  2.1318185  ... -0.4190606  -0.5247183
   0.07076679]]
MSE Train:  0.20987658
MSE Test:  0.20850624
pred:  [[ 1.0727872  -